# Anthropic Agent Skills
- https://www.anthropic.com/news/skills
- https://www.anthropic.com/engineering/equipping-agents-for-the-real-world-with-agent-skills
- Help: https://support.claude.com/en/articles/12512198-how-to-create-custom-skills
- Docs: https://docs.claude.com/en/docs/agents-and-tools/agent-skills/overview
- API: https://docs.claude.com/en/api/skills-guide
- Claude Code: https://docs.claude.com/en/docs/claude-code/skills
- GitHub: https://github.com/anthropics/skills

In [1]:
import anthropic # pip install anthropic
from anthropic.lib import files_from_dir
from dotenv import load_dotenv # pip install python-dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

### Create a Skill from a directory
- https://docs.claude.com/en/api/skills/create-skill

In [ ]:
client.beta.skills.create(
    betas=["skills-2025-10-02"],
    display_title="scf-brand",
    files=files_from_dir("scf-brand"),
)

### Create a Skill from a .zip file
- https://docs.claude.com/en/api/skills/create-skill

In [4]:
client.beta.skills.create(
    betas=["skills-2025-10-02"], # Beta header: https://docs.claude.com/en/api/skills-guide
    display_title="scf-brand",
    files=[("scf-brand.zip", open("scf-brand.zip", "rb"))],
)

SkillCreateResponse(id='skill_018asjGvKo57w4zAb2gf4fYH', created_at='2025-10-19T17:14:33.157571Z', display_title='scf-brand', latest_version='1760894072169934', source='custom', type='skill', updated_at='2025-10-19T17:14:33.157571Z')

### Using skills

In [5]:
response = client.beta.messages.create(
    model="claude-sonnet-4-5-20250929",
    max_tokens=16384, #4096, # https://docs.claude.com/en/api/messages#body-max-tokens
    #betas=["code-execution-2025-08-25", "skills-2025-10-02"],
    extra_headers={"anthropic-beta": "web-fetch-2025-09-10, code-execution-2025-08-25, skills-2025-10-02"}, # web fetch tool: https://docs.claude.com/en/docs/agents-and-tools/tool-use/web-fetch-tool
    container={
        "skills": [
            {
                "type": "anthropic",
                "skill_id": "pptx",
                "version": "latest"
            },
            {
                "type": "custom",
                "skill_id": "skill_018asjGvKo57w4zAb2gf4fYH", # From https://console.anthropic.com/workspaces/default/skills
                "version": "latest"
            }
        ]
    },
    messages=[{
        "role": "user",
        "content": "Create a presentation about Scandic Fusion. Use data from https://www.scandicfusion.com to get information about this company."
    }],
    tools=[{
        "type": "code_execution_20250825",
        "name": "code_execution"
    },
    {
        "type": "web_fetch_20250910",
        "name": "web_fetch",
        "max_uses": 5
    }]
)

**Total tokens in**: 401'860  
**Total tokens out**: 9'049  
**Total token cost**: 1.34 USD

In [ ]:
# Check if request was successful
if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

In [6]:
# Extract file IDs from the response
def extract_file_ids(response):
    file_ids = []
    for item in response.content:
        if item.type == 'bash_code_execution_tool_result':
            content_item = item.content
            if content_item.type == 'bash_code_execution_result':
                for file in content_item.content:
                    if hasattr(file, 'file_id'):
                        file_ids.append(file.file_id)
    return file_ids

In [7]:
extract_file_ids(response)

['file_011CUJ954aN4qcE7vqLrgbFB']

In [8]:
# Download the file using Files API
for file_id in extract_file_ids(response):
    file_metadata = client.beta.files.retrieve_metadata(
        file_id=file_id,
        betas=["files-api-2025-04-14"]
    )
    file_content = client.beta.files.download(
        file_id=file_id,
        betas=["files-api-2025-04-14"]
    )

    # Step 4: Save to disk
    file_content.write_to_file(file_metadata.filename)
    print(f"Downloaded: {file_metadata.filename}")

Downloaded: scandic_fusion_presentation.pptx


### Versions

In [6]:
from importlib.metadata import version
from IPython.display import Markdown, display # pip install ipython
import sys

packages = ['anthropic', 'python-dotenv']

text = f"Python version: {sys.version}\n\n"
for i in packages:
    text += f"[{i}](https://pypi.org/project/{i}/) version: {version(i)}\n\n"
display(Markdown(text))

Python version: 3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]

[anthropic](https://pypi.org/project/anthropic/) version: 0.71.0

[python-dotenv](https://pypi.org/project/python-dotenv/) version: 1.1.1

